In [2]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time

In [3]:
def createConnection(host, database, user, password):
    """
    Create a connection to the database
    """
    conn = psycopg2.connect(host=host,database=database,user=user,password=password)
    return conn

In [9]:
import datetime
from datetime import timedelta
conn = createConnection(host = 'ec2-3-211-228-251.compute-1.amazonaws.com', database = 'd3dll6bk435ijj', user = 'iiigxtlbnjbjat', password = '6fd22642e5a345a55e27b13eacddd8e7a40b3130ad5c4fc8b9bb002a9c98927c')
cursor = conn.cursor()

hour = (datetime.datetime.now() - timedelta(days=7)).strftime("%Y-%m-%dT%H")
print(hour)
sql_statement = """SELECT * FROM weather_forecast_hourly_v2 WHERE time_insert >= '{}'""".format(hour)
print(sql_statement)
df = pd.read_sql(sql_statement, conn)
conn.close()
df.info()


2021-12-13T16
SELECT * FROM weather_forecast_hourly_v2 WHERE time_insert >= '2021-12-13T16'
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   time_insert  2944 non-null   object 
 1   date_hour    2944 non-null   object 
 2   city         2944 non-null   object 
 3   temp         2944 non-null   float64
 4   precip       2944 non-null   float64
 5   wind_spd     2944 non-null   float64
 6   wind_dir     2944 non-null   float64
 7   rh           2944 non-null   float64
 8   snow         2944 non-null   float64
dtypes: float64(6), object(3)
memory usage: 207.1+ KB


In [10]:
import plotly.express as px

fig = px.line(df.loc[df['city'].isin(['New York','Buffalo','Bangalore','Austin','Los Angeles','Miami','Chicago'])], x='date_hour', y='temp', title='Temperature', color='city')


fig.update_layout(
    template = 'plotly_dark'
)
fig.show()

In [11]:
conn = createConnection(host = 'ec2-3-211-228-251.compute-1.amazonaws.com', database = 'd3dll6bk435ijj', user = 'iiigxtlbnjbjat', password = '6fd22642e5a345a55e27b13eacddd8e7a40b3130ad5c4fc8b9bb002a9c98927c')
cursor = conn.cursor()

sql_statement = """
    CREATE TABLE IF NOT EXISTS amazon_tweets_sentiment_v2 (
        id INTEGER PRIMARY KEY ,
        time TIMESTAMP,
        polarity REAL,
        mentions INTEGER,
        text TEXT,
        subjectivity REAL,
        user_location INTEGER,
        keyword TEXT
    );
"""

cursor.execute(sql_statement)
conn.commit()
conn.close()

In [12]:
import datetime
from datetime import timedelta

con = createConnection(host = '34.122.213.143', database = 'tweeper', user = 'postgres', password = 'aruba')
cursor = con.cursor()

hour = datetime.datetime.now() - timedelta(hours=24)
hour = hour.strftime("%Y-%m-%d %H:00:00")
sql_statement = f"""SELECT id,text, created_at, polarity, subjectivity, user_location 
                     FROM company_tweets 
                     WHERE created_at > '{hour}';"""

dats = pd.read_sql(sql_statement, con)
con.close()
dats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4698 entries, 0 to 4697
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             4698 non-null   object 
 1   text           4698 non-null   object 
 2   created_at     4698 non-null   object 
 3   polarity       4698 non-null   float64
 4   subjectivity   4698 non-null   float64
 5   user_location  3117 non-null   object 
dtypes: float64(2), object(4)
memory usage: 220.3+ KB


In [13]:
keys = ['Amazon','Netflix','Google','Apple']

for key in keys:
    if key == 'Amazon':
        amazon_df = dats[dats['text'].str.contains(key)]
    elif key == 'Netflix':
        netflix_df = dats[dats['text'].str.contains(key)]
    elif key == 'Google':
        google_df = dats[dats['text'].str.contains(key)]
    elif key == 'Apple':
        apple_df = dats[dats['text'].str.contains(key)]

display(amazon_df)


,id,text,created_at,polarity,subjectivity,user_location
6,1473013673710944265,RT @kingdomofwench: Because Amazon is jacking ...,2021-12-20 19:33:21,0.000000,0.000000,they/them/
7,1473013674512052224,RT @kingdomofwench: Because Amazon is jacking ...,2021-12-20 19:33:21,0.000000,0.000000,"sunnydale, ca"
8,1473013674998706187,RT @PS5Drop: Amazon Is Bound To Drop This Week...,2021-12-20 19:33:22,0.250000,0.500000,None
20,1473013687283638276,RT @shane_mcminn: FREE to download worldwide o...,2021-12-20 19:33:24,0.250000,0.450000,B.C. Canada
29,1473013675019624450,RT @Wario64: Mario + Rabbids Kingdom Battle (S...,2021-12-20 19:33:22,1.000000,0.300000,Gotham City
...,...,...,...,...,...,...
4671,1473016029362245634,RT @kingdomofwench: Because Amazon is jacking ...,2021-12-20 19:42:43,0.000000,0.000000,Your Dad's Ass
4672,1473016030146572297,- God kids getting got cards this year . \n\nG...,2021-12-20 19:42:43,0.000000,0.000000,None
4678,1473016031392280578,RT @Arkunir: Twitch qui ban Amazon : https://t...,2021-12-20 19:42:43,0.000000,0.000000,Kamogawa Boxing Gym
4695,1473016039445348359,"Walk the Narrow Way: How to be a Disciple, ava...",2021-12-20 19:42:45,0.211111,0.544444,None


In [15]:
amazon_df['created_at'] = pd.to_datetime(amazon_df['created_at'])
result = amazon_df.groupby(
    [pd.Grouper(key = 'created_at', freq = '600s'), 'polarity']
).count().unstack(fill_value=0).stack().reset_index()

result['polarity'] = result['polarity'].apply(lambda x: -1 if x < 0 else (1 if x > 0 else 0))
result = result.rename(columns = 
{'id': 'Count of Amazon mentions', 'created_at': 'Time'})
result['Keyword'] = 'Amazon'
time_series = result["Time"][result['polarity']==0].reset_index(drop=True)
fig = px.line(result, x = 'Time', y = 'Count of Amazon mentions', color = 'polarity', color_discrete_map = {0: 'red', 1: 'green'})
fig.show()

/var/folders/yg/xh1sg6gx0t9dmrk3m0704ppc0000gn/T/ipykernel_10541/1547474846.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [16]:
inserts = []

for item in result.itertuples():
    insert = (item[0],item[1], item[2], item[3], item[4], item[5], item[6], item[7])
    inserts.append(insert)

#print(inserts)

sql_statement = """
    INSERT INTO amazon_tweets_sentiment_v2 (id, time, polarity, mentions, text, subjectivity, user_location, keyword)
    VALUES (%s,%s, %s, %s, %s, %s, %s, %s)
"""
try:
    conn = createConnection(host = '34.122.213.143', database = 'tweeper', user = 'postgres', password = 'aruba')
    cursor = conn.cursor()
    cursor.executemany(sql_statement, inserts)
    conn.commit()
except Exception as e:
    print(e)
    conn.rollback()
conn.close()

duplicate key value violates unique constraint "amazon_tweets_sentiment_v2_pkey"
DETAIL:  Key (id)=(0) already exists.



In [19]:
import re
content = ''.join(amazon_df['text'])
content = re.sub(r"http\S+", "", content)
content = content.replace('RT ', ' ').replace('&amp;', 'and')
content = re.sub('[^A-Za-z0-9]+', ' ', content)
content = content.lower()

In [20]:
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

tokenized = word_tokenize(content)
stop_words = set(stopwords.words('english'))
filtered_sentence = [w for w in tokenized if not w in stop_words]


fdist = FreqDist(filtered_sentence)
fd = pd.DataFrame(fdist.most_common(10), columns = ['Word', 'Frequency'])


In [21]:
fd['date_hour'] = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M")
display(fd)

,Word,Frequency,date_hour
0,amazon,535,2021-12-20T16:45
1,99,63,2021-12-20T16:45
2,12,61,2021-12-20T16:45
3,gift,56,2021-12-20T16:45
4,sell,53,2021-12-20T16:45
5,thread,50,2021-12-20T16:45
6,kingdomofwench,49,2021-12-20T16:45
7,jacking,49,2021-12-20T16:45
8,prices,49,2021-12-20T16:45
9,known,49,2021-12-20T16:45


In [22]:
inserts = []

for item in fd.itertuples():
    inserts.append((item[1], item[2], item[3]))

sql_statement = """
    INSERT INTO amazon_tweets_fd (word, frequency, date_hour)
    VALUES (%s, %s, %s)"""

try:
    conn = createConnection(host = 'ec2-3-211-228-251.compute-1.amazonaws.com', database = 'd3dll6bk435ijj', user = 'iiigxtlbnjbjat', password = '6fd22642e5a345a55e27b13eacddd8e7a40b3130ad5c4fc8b9bb002a9c98927c')
    cursor = conn.cursor()  
    cursor.executemany(sql_statement, inserts)
    conn.commit()
    cursor.close()
except Exception as e:
    print(e)
    conn.rollback()

In [23]:
fig = px.bar(fd, x = 'Word', y = 'Frequency')
fig.show()

In [24]:
import itertools

states = ['Alabama', 'AL', 'Alaska', 'AK', 'Arizona', 'AZ', 'Arkansas', 'AR', 'California', 'CA', 'Colorado', 'CO', 'Connecticut', 'CT', 'Delaware', 'DE', 'Florida', 'FL', 'Georgia', 'GA', 'Hawaii', 'HI', 'Idaho', 'ID', 'Illinois', 'IL', 'Indiana', 'IN', 'Iowa', 'IA', 'Kansas', 'KS', 'Kentucky', 'KY', 'Louisiana', 'LA', 'Maine', 'ME', 'Maryland', 'MD', 'Massachusetts', 'MA', 'Michigan', 'MI', 'Minnesota', 'MN', 'Mississippi', 'MS', 'Missouri', 'MO', 'Montana', 'MT', 'Nebraska', 'NE', 'Nevada', 'NV', 'New Hampshire', 'NH', 'New Jersey', 'NJ', 'New Mexico', 'NM', 'New York', 'NY', 'North Carolina', 'NC', 'North Dakota', 'ND', 'Ohio', 'OH', 'Oklahoma', 'OK', 'Oregon', 'OR', 'Pennsylvania', 'PA', 'Rhode Island', 'RI', 'South Carolina', 'SC', 'South Dakota', 'SD', 'Tennessee', 'TN', 'Texas', 'TX', 'Utah', 'UT', 'Vermont', 'VT', 'Virginia', 'VA', 'Washington', 'WA', 'West Virginia', 'WV', 'Wisconsin', 'WI', 'Wyoming', 'WY']
state_dict = dict(itertools.zip_longest(*[iter(states)] *2 , fillvalue=""))
inv_state_dict = dict((v,k) for k,v in state_dict.items())

print(state_dict)

{'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}


In [25]:
in_us = []
loc = amazon_df[['user_location']]
amazon_df = amazon_df.fillna(' ')

for item in amazon_df['user_location']:
    check = False
    for state in states:
        if state in item:
            in_us.append(state_dict[state] if state in state_dict else state)
            check = True
            break
    if not check:
        in_us.append(None)

geo_dist = pd.DataFrame(in_us, columns = ['State']).reset_index()

In [26]:
geo_dist = geo_dist.groupby('State').count()
geo_dist = geo_dist.rename(columns = {'index': 'Count'})
geo_dist = geo_dist.sort_values(by = 'Count', ascending = False).reset_index()

geo_dist['date_hour'] = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M")


In [31]:
geo_dist['State Name'] = geo_dist['State'].apply(lambda x: inv_state_dict[x] if x in inv_state_dict else x)
geo_dist['text'] = geo_dist['State Name'] + ': ' + geo_dist['Count'].astype(str)


In [32]:
geo_dist

,State,Count,date_hour,State Name,text
0,CA,11,2021-12-20T16:47,California,California: 11
1,NY,11,2021-12-20T16:47,New York,New York: 11
2,TX,8,2021-12-20T16:47,Texas,Texas: 8
3,FL,7,2021-12-20T16:47,Florida,Florida: 7
4,OR,5,2021-12-20T16:47,Oregon,Oregon: 5
5,MD,5,2021-12-20T16:47,Maryland,Maryland: 5
6,OH,5,2021-12-20T16:47,Ohio,Ohio: 5
7,NC,4,2021-12-20T16:47,North Carolina,North Carolina: 4
8,MA,4,2021-12-20T16:47,Massachusetts,Massachusetts: 4
9,CO,4,2021-12-20T16:47,Colorado,Colorado: 4


In [33]:
inserts = []

for item in geo_dist.itertuples():
    inserts.append((item[3], item[2], item[1]))

sql_statement = """
    INSERT INTO amazon_tweets_geo (date_hour, count, state)
    VALUES (%s, %s, %s)"""


try:
    conn = createConnection(host = 'ec2-3-211-228-251.compute-1.amazonaws.com', database = 'd3dll6bk435ijj', user = 'iiigxtlbnjbjat', password = '6fd22642e5a345a55e27b13eacddd8e7a40b3130ad5c4fc8b9bb002a9c98927c')
    cursor = conn.cursor()  
    cursor.executemany(sql_statement, inserts)
    conn.commit()
    cursor.close()
except Exception as e:
    print(e)
    conn.rollback()

duplicate key value violates unique constraint "amazon_tweets_geo_pkey"
DETAIL:  Key (date_hour, state)=(2021-12-20T16:47, CA) already exists.



In [36]:
import plotly.graph_objects as go

fig = go.Figure(data = go.Choropleth(
    locations = geo_dist['State'],
    z = geo_dist['Count'].astype(float),
    locationmode = 'USA-states',
    colorscale = 'Greens',
    text = geo_dist['text'],
    marker_line_color = 'black',
    marker_line_width = 0.5,
    colorbar_title = 'Count'
))

fig.update_layout(geo_scope = 'usa')
fig.show()

In [35]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows = 2, cols = 2,
    column_widths = [1,0.4],
    row_heights = [0.6,0.4],
    specs = [[{'type': 'scatter', 'rowspan' : 2}, {'type': 'choropleth'}],[None, {'type': 'bar'}]]
)

fig.add_trace(go.Scatter(
    x = time_series,
    y = result['Count of Amazon mentions'][result['polarity']==0].reset_index(drop=True),
    name = 'Neutral',
    opacity = 0.8), row = 1, col = 1
)

fig.add_trace(go.Scatter(
    x = time_series,
    y = result['Count of Amazon mentions'][result['polarity']==1].reset_index(drop=True),
    name = 'Positive',
    opacity = 0.8), row = 1, col = 1
)

fig.add_trace(go.Scatter(
    x = time_series,
    y = result['Count of Amazon mentions'][result['polarity']==-1].reset_index(drop=True),
    name = 'Negative',
    opacity = 0.8), row = 
    1, col = 1
)

fig.add_trace(go.Bar(
    x = fd['Word'],
    y = fd['Frequency'],
    name = 'Frequency'), row = 2, col = 2
)

fig.update_traces(marker_color='rgb(59, 89, 152)',          
    marker_line_color='rgb(8,48,107)',                      
    marker_line_width=0.5, opacity=0.7, row=2, col=2)

fig.add_trace(go.Choropleth(
    locations = geo_dist['State'],
    z = geo_dist['Count'].astype(float),
    locationmode = 'USA-states',
    colorscale = 'Reds',
    text = geo_dist['text'],
    marker_line_color = 'black',
    marker_line_width = 0.5,
    showscale = False,
    geo = 'geo',
    colorbar_title = 'Count'), row = 1, col = 2
    
)

fig.update_layout(
    title_text =  "Real-time tracking for '{}' mentions on Twitter ".format('Amazon',datetime.datetime.utcnow().strftime('%m-%d %H:%M') ),
    geo = dict(
        scope='usa',
    ),
    template="plotly_dark",
    margin=dict(r=20, t=50, b=50, l=20),
    annotations=[
        go.layout.Annotation(
            text="Data Source: Twitter",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0,
            y=0)
    ],
    showlegend=True,
    xaxis_rangeslider_visible=True
)

fig.show()